## *RISK* attrition

[StackExchange question.](https://math.stackexchange.com/questions/4318008/comparing-the-probability-of-parallel-multiple-dice-rolls)

* Two sides each have a pool of d6s.
* Each side rolls their dice, and then pairs of one die from each side are made from highest to lowest.
    (Some dice may be unpaired.)
* For each pair, if one side rolled a higher number, they eliminate the opposing die.
    On a tie nothing happens to that pair.
* Repeat until one side runs out of dice.

In [1]:
import piplite
await piplite.install("hdroller")

import hdroller

class EvalRiskAttrition(hdroller.EvalPool):
    def next_state(self, state, outcome, a, b):
        if state is None:
            score_a, score_b, advantage = 0, 0, 0
        else:
            score_a, score_b, advantage = state
        # Advantage is the number of unpaired dice that rolled a previous (higher) number.
        # If positive, it favors side A, otherwise it favors side B.
        # We pair them off with newly-rolled dice of the disadvantaged side.
        if advantage > 0:
            score_a += min(b, advantage)
        elif advantage < 0:
            score_b += min(a, -advantage)
        advantage += a - b
        return score_a, score_b, advantage
    
    def final_outcome(self, final_state, pool_a, pool_b):
        score_a, score_b, advantage = final_state
        if score_a == 0 and score_b == 0:
            # No change. Eliminate this outcome to prevent infinite looping.
            return hdroller.Reroll
        # Each side loses dice equal to the other's hits.
        # The result is the number of remaining dice on each side.
        return pool_a.num_dice() - score_b, pool_b.num_dice() - score_a
    
    def direction(self, *_):
        # See outcomes in descending order.
        return -1

eval_risk = EvalRiskAttrition().bind_dice(hdroller.d6, hdroller.d6)

def risk_attrition(a, b):
    if a == 0 or b == 0:
        # If one side has run out of dice, no more rolling is necessary.
        return a, b
    else:
        # Otherwise, run the contest.
        return eval_risk(a, b)

# 4 dice vs. 3 dice.
a = 4
b = 3
print(hdroller.Die((a, b)).sub(risk_attrition, max_depth=None, denominator_method='reduce'))

Denominator: 350190883979307611136000
| Outcome[0] | Outcome[1] |                   Weight | Probability |
|-----------:|-----------:|-------------------------:|------------:|
|          0 |          1 |  11215262070269292175045 |   3.202614% |
|          0 |          2 |  26208104640905472978960 |   7.483948% |
|          0 |          3 |  44739780296050462334400 |  12.775827% |
|          1 |          0 |  11215262070269292175045 |   3.202614% |
|          2 |          0 |  28634396560615778884510 |   8.176797% |
|          3 |          0 |  60745546693229402315592 |  17.346410% |
|          4 |          0 | 167432531647967910272448 |  47.811790% |



### Versus Monte Carlo

We can compare this to a Monte Carlo simulation, which is slower and noisier, but provides an independent alternative.

In [2]:
from collections import Counter

def monte_carlo(a, b):
    if a == 0 or b == 0:
        return a, b
    else:
        rolls_a = sorted((hdroller.d6.sample() for i in range(a)), reverse=True)
        rolls_b = sorted((hdroller.d6.sample() for i in range(b)), reverse=True)
        for roll_a, roll_b in zip(rolls_a, rolls_b):
            if roll_a > roll_b: b -= 1
            if roll_b > roll_a: a -= 1
        return monte_carlo(a, b)

counts = Counter()

for i in range(10000):
    counts[monte_carlo(a, b)] += 1

for key in sorted(counts.keys()):
    print(key, counts[key])


(0, 1) 331
(0, 2) 759
(0, 3) 1238
(1, 0) 312
(2, 0) 822
(3, 0) 1782
(4, 0) 4756
